# Assignment 3 Info


## Class Notes
- Use BeautifulSoup (4) for html scraping
  - https://realpython.com/python-web-scraping-practical-introduction/
  - https://realpython.com/beautiful-soup-web-scraper-python/#step-1-inspect-your-data-source
  - `.prettify()` can be useful to look at the source code
- To join absolute and reltative pages: use module `urlparse`
  - `urlparse.urljoin(url1, url2)`
- To actually *get* the page, you should use the module `requests`
  - `page = requests.get(URL)`
  - `soup = BeautifulSoup(page.content , "html.parser"`
- Process:
  - How ma

## Summary
- **Deadline**: 1st April 2022 18:00
-  Questions:
    1. Are there couples among the employees. If so, who? Are they still together?
       1. Look if they go on holidays together
       2. Look if they changed surnames
       3. Look if they had children
    2. Did any of the employees have a child? If so, who?
       1. 9+ month "break"
       2. If the mother leaves for 9+ months, then the father might also take a leave
       3. They might decrease how much they work (frequency of publish)
    3. If you would be looking to work for Tabularazor Inc., how many holidays can you expect to get per year?
       1. Look at all periods that are larger than "working but not publishing", but shorter than "we're having a baby"

## Group 14
- Antonio Sanchez Martin - ''5245834''
- Felix Unger - ''5673631''
- Jeroen van Paassen - ''4720970''
- Yunus Emre Torlak - ''5597668''

## Planning:

1. Put the code in visual studio and visualize the basic dataset
2. Figure out how to use the dataset to do the plots
3. Make the plots (draw conslusions?)
4. How do we use the plots to answer the questions?
5. Structure the report, divide the questions and explain relevant plots

## Plots

First, we plot some diagrams to visualize the dataset that we have. The plots that we will make are:
- histogram with number of employees on x-axis and amount of work posted on the y-axis (to see frequency of work posted)
- a plot with the productivity of a random employee over time
- histogram with amount of employees on the x-axis and the frequency of reoccuring last name on the y-axis
- the productivity of employees sharing the same name over time? (covariance)
- scatterplot with years on x-axis and time on y-axis, to see the behaviour of posting articles over time

## Notes

Some important things that we need to take into account when using the dataset:
- there could be people with the exact same name
- an absence of over 16 weeks could be considered as pregnancy (6 for pregnancy and 10 after giving birth)
- for men that is at minimum 1 week (->can we link ''holidays'' to pregnancy of the partner?)

## Answering the questions

After making the plots, we focus on answering the questions, which we will do with the help of the plots:
a. Are there couples among the employees. If so, who? Are they still together?
- same/change of last name, covariance in days of, corresponding time of absence (partner of pregnant employee is absent for a short while during the 16+ week absence of the female partner)
b. Did any of the employees have a child? If so, who?
- absence for 16+ weeks, decrease in productivity, change in posting behaviour
c. If you would be looking to work for Tabularazor Inc., how many holidays can you expect to get per
year?
- look at average days off/productivity, account for pregnancy

# Code

## Making the table
- Row -> Article entry on the website
- Columns -> 
  - Name
  - Date
  - Time
- Idea: Use `datetime` for storing the date data in a flexible format.

In [1]:
website = 'https://jdestefani.github.io/SEN163A-TabularRazorArchives/'

In [2]:
import requests
import bs4 as bs

get_soup = lambda link: bs.BeautifulSoup(requests.get(link).content)
soup = get_soup(website)
print(soup.prettify())

<html>
 <head>
  <title>
   Articles
  </title>
 </head>
 <body>
  <h1>
   All articles
  </h1>
  <br/>
  <br/>
  <h2>
   Articles year 2012
  </h2>
  <br/>
  <div class="yearlink">
   <a href="./2012.html">
    Articles in 2012
   </a>
  </div>
  <h2>
   Articles year 2013
  </h2>
  <br/>
  <div class="yearlink">
   <a href="./2013.html">
    Articles in 2013
   </a>
  </div>
  <h2>
   Articles year 2014
  </h2>
  <br/>
  <div class="yearlink">
   <a href="./2014.html">
    Articles in 2014
   </a>
  </div>
  <h2>
   Articles year 2015
  </h2>
  <br/>
  <div class="yearlink">
   <a href="./2015.html">
    Articles in 2015
   </a>
  </div>
  <h2>
   Articles year 2016
  </h2>
  <br/>
  <div class="yearlink">
   <a href="./2016.html">
    Articles in 2016
   </a>
  </div>
  <h2>
   Articles year 2017
  </h2>
  <br/>
  <div class="yearlink">
   <a href="./2017.html">
    Articles in 2017
   </a>
  </div>
  <h2>
   Articles year 2018
  </h2>
  <br/>
  <div class="yearlink">
   <a href="./

In [3]:
# Get all links on site
# Checked: Works as expected
from urllib.parse import urljoin

def get_links(to_scrape = 'https://jdestefani.github.io/SEN163A-TabularRazorArchives/'):
    soup = get_soup(to_scrape)
    links = soup.find_all('a', href=True)
    if links is False:
        return [] # remember that empty arrays are "false" in python
    links = [urljoin(to_scrape, link['href']) for link in links]
    return links

print(get_links(to_scrape=website))

['https://jdestefani.github.io/SEN163A-TabularRazorArchives/2012.html', 'https://jdestefani.github.io/SEN163A-TabularRazorArchives/2013.html', 'https://jdestefani.github.io/SEN163A-TabularRazorArchives/2014.html', 'https://jdestefani.github.io/SEN163A-TabularRazorArchives/2015.html', 'https://jdestefani.github.io/SEN163A-TabularRazorArchives/2016.html', 'https://jdestefani.github.io/SEN163A-TabularRazorArchives/2017.html', 'https://jdestefani.github.io/SEN163A-TabularRazorArchives/2018.html', 'https://jdestefani.github.io/SEN163A-TabularRazorArchives/2019.html']


In [6]:
# Start getting url
import pandas as pd
import asyncio

class Scraper:
    def __init__(self, website:str):
        self.scraped_data = list() # create a list to append all website inforation
        self.scrape(website)
        fields = ['link','author', 'date', 'time'] # columns of table
        self.scraped_data= pd.DataFrame(self.scraped_data, columns = fields) # export as a dataframe

    def scrape(self, website): 
        all_links = get_links(website)
        if all_links:
            # Keep on searching
            for link in all_links:
                ## print(f"Searching: {link}")
                self.scrape(link)
        else:
            # Get the data (i.e. base case)
            ## print("Reached the bottom")
            soup = get_soup(website) # get the soup object for extracting info
            web_info= list()
            web_info.append(website)
            web_info.append(soup.find_all('div', class_='author')[0].text)
            web_info.append(soup.find_all('div', class_='date')[0].text)
            web_info.append(soup.find_all('div', class_='time')[0].text)
            self.scraped_data.append(web_info) # append to object list
            
# output = Scraper(website)
output = Scraper('https://jdestefani.github.io/SEN163A-TabularRazorArchives/2012-1.html').scraped_data

# Save the data so that it does not have to be scraped every time
output.to_csv('scraped_data.csv', index=False)


In [7]:
# Load the scraped web data
import pandas as pd
web_data = pd.read_csv('scraped_data.csv')
web_data.head()


,link,author,date,time
0,https://jdestefani.github.io/SEN163A-TabularRa...,Jaye Shimek,2012-01-01,15:17


# Plots to be done

### Scatterplot
Axis Formatting:
-y-axis Timestamp with 0-24 h 
-x-axis date from 2012 to 2020
-different colors each employee

Analysis: 
-changes in publishing time and frequency? 
-correlation to children or marriage? 




### Employee Histogram
Axis Formatting: histogram with number of employees on x-axis and amount of work posted on the y-axis (to see frequency of work posted), 
note: if productivity is double the average, it could be the case that there are 2 employees with the exact same name

    

### Last Name Histogram
